In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import pickle
from pickle import dump

In [2]:
# Read in csv
df = pd.read_csv("LouisvilleCleanFinal.csv").drop(['Unnamed: 0'],axis=1)
df.head()

,PrimaryColor,BreedCategory,IntakeStatus,PetAgeCategory,Sex,TopBreed,BinaryOutcome
0,WHITE,PURE,NORMAL,YOUNG,MALE,PIT BULL TERRIER,DENY
1,BLACK,MIX,NORMAL,YOUNG,FEMALE,BORDER COLLIE,TAKE
2,TAN,MIX,NORMAL,YOUNG,MALE,GOLDEN RETRIEVER,TAKE
3,WHITE,PURE,NORMAL,BABY,FEMALE,LABRADOR RETRIEVER,TAKE
4,BLACK,PURE,SICK,YOUNG,MALE,PIT BULL TERRIER,DENY


In [14]:
df.groupby(df['BinaryOutcome']).count()
print(13135/(13135+28400))

0.3162393162393162


In [4]:
df["BinaryOutcome2"] = 0
df.loc[df["BinaryOutcome"].str.contains("TAKE"), "BinaryOutcome2"] = 1

In [5]:
df.head()

,PrimaryColor,BreedCategory,IntakeStatus,PetAgeCategory,Sex,TopBreed,BinaryOutcome,BinaryOutcome2
0,WHITE,PURE,NORMAL,YOUNG,MALE,PIT BULL TERRIER,DENY,0
1,BLACK,MIX,NORMAL,YOUNG,FEMALE,BORDER COLLIE,TAKE,1
2,TAN,MIX,NORMAL,YOUNG,MALE,GOLDEN RETRIEVER,TAKE,1
3,WHITE,PURE,NORMAL,BABY,FEMALE,LABRADOR RETRIEVER,TAKE,1
4,BLACK,PURE,SICK,YOUNG,MALE,PIT BULL TERRIER,DENY,0


In [6]:
# Define X and y
X = df.drop(['BinaryOutcome','BinaryOutcome2'], axis="columns")
y = df[['BinaryOutcome2']]


In [7]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
X = ohe.fit_transform(X)

In [8]:
y.shape

(41535, 1)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Import, initialize, fit and predict
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators = 100, oob_score = 'True', n_jobs = -1,random_state =50, max_features = "auto", min_samples_leaf = 50)
rf_model.fit(X_train, y_train)

# Test
predict_y_test = rf_model.predict(X_test)


C:\Users\Lenovo\Anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys


#### Evaluating

In [10]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix

In [11]:
# Validate - run accuracy and AUC score
from sklearn import metrics
print("Accuracy score: ", metrics.accuracy_score(y_test, predict_y_test))
print("AUC score: ", metrics.roc_auc_score(y_test, predict_y_test))

Accuracy score:  0.7048343605546995
AUC score:  0.5834387757272487


In [12]:
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, predict_y_test))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, predict_y_test))
print('\n')

=== Confusion Matrix ===
[[6492  595]
 [2470  827]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.72      0.92      0.81      7087
           1       0.58      0.25      0.35      3297

    accuracy                           0.70     10384
   macro avg       0.65      0.58      0.58     10384
weighted avg       0.68      0.70      0.66     10384





#### Tree nodes and height

In [13]:
from rfpimp import *
print(f"{rfnnodes(rf_model):,d} tree nodes and {np.median(rfmaxdepths(rf_model))} median tree height")

24,046 tree nodes and 20.0 median tree height


C:\Users\Lenovo\Anaconda3\envs\pythondata\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


#### Feature importance


In [ ]:
# importance = rf_model.feature_importances_
# # summarize feature importance
#recall_score(y_true, y_pred, average=None, zero_division=1)

#### Cross val score

In [ ]:
from sklearn.model_selection import cross_val_score

# More efficient use of data (every observation is used for train/test)
# But difficult to inspect results with either confusion matrix/roc curve
rf_cv_score = cross_val_score(rf_model, X, y, cv=10, scoring='accuracy')


In [ ]:
print("=== All Accuracy Scores ===")
print(rf_cv_score)
print('\n')
print("=== Mean Accuracy Score ===")
print("Mean Accuracy Score - Random Forest: ", rf_cv_score.mean())

In [ ]:


roc_auc_score(y_test, predict_y_test)


### Saving the model



In [ ]:

# dump(rf_model, open('rf_modelTESTMarc.pkl', 'wb'))
# new_input = [[0, 1, 0, 0, 1, 0]]
# with open('rf_modelTESTMarc.pkl', 'rb') as file:  
#     model = pickle.load(file)
# model.predict(new_input) 

In [ ]:
# pd.read_pickle('rf_modelTEST2.pkl')

In [ ]:
# # To run SVM model
# # Need to create 0/1 for BinaryOucome
# df["BinaryOutcome2"] = 0
# df.loc[df["BinaryOutcome"].str.contains("TAKE"), "BinaryOutcome2"] = 1

# # Define X and y
# X = df.drop(['BinaryOutcome','BinaryOutcome2'], axis="columns")
# y = df['BinaryOutcome2']

# # One Hot encode X
# from sklearn.preprocessing import OneHotEncoder
# ohe = OneHotEncoder()
# X = ohe.fit_transform(X).toarray()


# # Split
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X ,y, random_state=20) 

# # Train
# from sklearn.svm import SVC 
# svm_model = SVC(kernel='rbf')
# svm_model.fit(X_train, y_train)

# # Model Accuracy
# print('Test Acc: %.4f' % svm_model.score(X_test, y_test))

# # Save 
# pickle.dump(svm_model, open("SVMmodelonehot.pkl","wb"))